Tape measure
aruco tags
tripod
checkboard

Steps: 
Get a video - Together
Calibration - Satchel
Create a mask for finding ball - Zaraius
Create a yolo model for finding ball -  Satchel
Use mask for find position - Zaraius
Use bounding box for finding position - Zaraius
Connect frame by frame position to get velocity (SORT) - Satchel

Find size to depth research
hough transform
Once you've detected center of the ball, edge segmentation or line detector

In [ ]:
# Calibration
distance = []
ball_diam = []

In [ ]:
# Process all calibration photos
calibration_dir = 'distance_calibration_photos'
ball_measurer = BallMeasurement()

# Lists to store measurements
distances = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11.66]) * (11.5/12)  # in feet
diameters = [314.78, 262.03, 188.0, 164.15, 143.57, 119.33, 106.08, 98.08, 86.47, 81.58, 79.10]  # in pixels

# Process each image
for filename in sorted(os.listdir(calibration_dir)):
    if filename.endswith('.jpeg'):
        # Extract distance from filename (assuming format like '10ft.jpeg')
        distance = float(filename.replace('ft.jpeg', ''))
        
        # Full path to image
        image_path = os.path.join(calibration_dir, filename)
        
        print(f"\nProcessing image at {distance} feet...")
        diameter = ball_measurer.measure_ball(image_path)
        
        if diameter is not None:
            distances.append(distance)
            diameters.append(diameter)
            print(f"Measured diameter: {diameter:.2f} pixels")

# Print results
print("\nCalibration Results:")
for dist, diam in zip(distances, diameters):
    print(f"Distance: {dist}ft, Ball diameter: {diam:.2f} pixels")

# Save results to lists for later use
distance = distances
ball_diam = diameters

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
distances = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11.66]) * (11.5/12)  # in feet
diameters = [314.78, 262.03, 188.0, 164.15, 143.57, 119.33, 106.08, 98.08, 86.47, 81.58, 79.10]  # in pixels

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(distances, diameters, color='blue', label='Measurements')
plt.plot(distances, diameters, 'b--', alpha=0.5)

# Add labels and title
plt.xlabel('Distance (feet)')
plt.ylabel('Ball Diameter (pixels)')
plt.title('Ball Size vs Distance Calibration')
plt.grid(True, alpha=0.3)
plt.legend()

# Add a trend line to help visualize the relationship
z = np.polyfit(distances, diameters, 2)
p = np.poly1d(z)
x_trend = np.linspace(min(distances), max(distances), 100)
plt.plot(x_trend, p(x_trend), 'r-', alpha=0.7, label=f'Trend Line (degree=2)')
plt.legend()

plt.show()

# Print the polynomial coefficients for future reference
print("\nQuadratic fit coefficients (ax² + bx + c):")
print(f"a = {z[0]:.4f}")
print(f"b = {z[1]:.4f}")
print(f"c = {z[2]:.4f}")

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt") # Load a nano-sized YOLOv11 model
results = model.predict("distance_calibration_photos/11ft.jpg")
for r in results:
    im_array = r.plot()  # plot returns an image with detections
    im = Image.fromarray(im_array[..., ::-1])  # Convert to RGB PIL image
    im.show() # Display the image


In [ ]:
from segmentation import BasketballSegmentation
segmentation = BasketballSegmentation("best2.pt", "distance_calibration_photos", "distance_output")
radii = segmentation.process_folder()
print(radii)


distances = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11.66]) * (11.5/12)  # in feet
diameters = [314.78, 262.03, 188.0, 164.15, 143.57, 119.33, 106.08, 98.08, 86.47, 81.58, 79.10]  # in pixels
